# Desafio 2 - PMP LPT/SPT

**Integrantes:**

* Lucas Hideki Takeuchi Okamura NUSP: 9274315

* Thales Arantes Kerche Nunes NUSP: 10769372

**Objetivos**

* Minimizar o tempo total (makespan) na programação de máquinas paralelas (PMP - Parallel Machine Problem), utilizando a regra do maior tempo de processamento (LPT - longest processing time)

**Descrição**
* Dado um conjunto de ‘n’ jobs a serem processados em uma dentre ‘m’ máquinas, alocar os jobs nas máquinas de modo a minimizar o tempo total de conclusão dos jobs (makespan)

* Alocados os jobs nas máquinas, ordenar os jobs em cada máquina pelo menor tempo de processamento (SPT – Shortest Processing Time) para minimizar o tempo médio de fluxo (Fbar)

In [1]:
# Importando libraries necessárias
import pandas as pd

# 1. Criando Classes de interesse

In [2]:
class Job:
    def __init__(self,i,pi):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.C=0.0     # completion time
        self.T=0.0     # tardiness

In [3]:
class Maq:
    def __init__(self,k):
        self.k=k     # índice da máquina
        self.jobs=[]     # jobs alocados à máquina
        self.carga=0.0     # carga alocada à máquina

# 2. Definindo funções auxiliares

A função _read_sheet_   lê uma planilha com dados de teste e devolve um Data Frame com os jobs, o número de jobs e o número de máquinas. Os argumentos de entrada são o nome do arquivo e o nome da aba / planilha.

In [4]:
def read_sheet(arc_name, sheet_name):
    n_m_prov = pd.read_excel(arc_name, sheet_name = sheet_name, header = None, index_col = 0)
    n = n_m_prov.loc['n', 1]
    m = n_m_prov.loc['m', 1]
    
    jobs = pd.read_excel(arc_name, sheet_name = sheet_name, skiprows = 9)
    
    return n, m, jobs

A função _save_sheet_  salva a planilha em um arquivo Excel com o nome escolhido. 

In [5]:
def save_sheet(arc_name, sheet_name, maqs, Cmax, Fbar, writer):
    df = []
    print('{:2s} {:2s} {:2s} {:2s}'.format('maq','#','p','C'))
    for maq in maqs:
        for job in maq.jobs:
            df.append([maq.k,job.i, job.p, job.C])
            print('{:2d} {:2d} {:2d} {:2d}'.format(maq.k, job.i, job.p, job.C))
    print()
    print('Cmax: {:4.1f}'.format(Cmax))
    print('Fbar: {:4.1f}'.format(Fbar))
    
    df = pd.DataFrame(df, columns = ['maq', 'job', 'p', 'C'])
    data = pd.DataFrame([[Cmax, Fbar]], columns = ['Cmax', 'Fbar'])
    
    df.to_excel(writer, sheet_name = sheet_name, startrow = 10, index = False)
    data.to_excel(writer, sheet_name = sheet_name, startrow = 5, index = False)

A função _create_jobs_ faz a criação de uma lista de jobs a partir do número de jobs "n" e uma lista de tempos de execução "p".

In [6]:
def create_jobs(n, p):    # n é o número de jobs e p é a lista com os tempos de execução
    jobs=[]
    for i in range(n):
        jobs.append(Job(i,p[i]))

    return jobs

A função _create_maq_ cria uma lista de máquinas a partir de um valor "m", que é o próprio número de máquinas.

In [7]:
def create_maqs(m):
    maqs=[]
    for k in range(m):
        maqs.append(Maq(k))
        
    return maqs

# 3. Definindo função principal PMP LPT-SPT

**Objetivo**

* Minimizar o tempo total (makespan) na programação de máquinas paralelas (PMP - Parallel Machine Problem), utilizando a regra do maior tempo de processamento (LPT - longest processing time)

**Descrição**

* Dado um conjunto de ‘n’ jobs a serem processados em uma dentre ‘m’ máquinas, alocar os jobs nas máquinas de modo a minimizar o tempo total de conclusão dos jobs (makespan)

* Alocados os jobs nas máquinas, ordenar os jobs em cada máquina pelo menor tempo de processamento (SPT – Shortest Processing Time) para minimizar o tempo médio de fluxo (Fbar)

**Procedimento**

* Passo 1: Ordenar os jobs em ordem decrescente dos tempos de processamento (LPT).

* Passo 2: Seguindo a ordem LPT, alocar os jobs na máquina com menor carga (soma dos tempos de processamento dos jobs já alocados). Em caso de empate, escolher arbitrariamente uma entre as máquinas com menor carga (p.ex: máquina de menor índice).

* Passo 3: Definidas as alocações, ordenar os jobs em cada máquina em ordem crescente dos tempos (SPT).

* Passo 4: Calcular os indicadores Cmax e Fbar.

In [8]:
def PMP(jobs, maqs):
    jobs.sort(key = lambda job: job.p, reverse = True)
    for job in jobs:
        minimun = 999999999
        for j in range(len(maqs)):
            if maqs[j].carga < minimun:
                minimun = maqs[j].carga
                ind_min = j
        maqs[ind_min].carga += job.p
        maqs[ind_min].jobs.append(job)
    
    Cmax=0
    for maq in maqs:
        maq.jobs.sort(key=lambda job: job.p)
        maq.jobs[0].C=maq.jobs[0].p
        for j in range(1,len(maq.jobs)):
            maq.jobs[j].C=maq.jobs[j-1].C+maq.jobs[j].p
        if maq.carga > Cmax:
            Cmax = maq.carga
        
    Fbar = sum([job.C for maq in maqs for job in maq.jobs])/n
                
    return jobs, maqs, Cmax, Fbar           

# 4. Testes e Gravação em Excel

In [9]:
with pd.ExcelWriter("Desafio 2 - PMP LPT - Dupla 09.xlsx", engine = 'xlsxwriter') as writer:
    for i in range(1, 8):
        print("Teste {}".format(i))
        n, m, jobs = read_sheet(arc_name = "xl06 3 A PMP Cmax LPT-SPT.xlsx", sheet_name = "I({})".format(i))
        jobs = create_jobs(n, list(jobs.p))
        maqs = create_maqs(m)
        jobs_final, maqs_final, Cmax_final, Fbar_final = PMP(jobs, maqs)
        save_sheet(arc_name = "Desafio 2 - PMP LPT - Dupla 09.xlsx", sheet_name = "teste {}".format(i),
                    maqs = maqs_final, Cmax = Cmax_final, Fbar = Fbar_final, writer = writer)
        print()

Teste 1
maq #  p  C 
 0  2  2  2
 0  3  6  8
 1  4  1  1
 1  0  3  4
 1  1  4  8

Cmax:  8.0
Fbar:  4.6

Teste 2
maq #  p  C 
 0  5  1  1
 0  0  4  5
 0  4 15 20
 1  7  1  1
 1  1  2  3
 1  6  5  8
 1  3 12 20
 2  8  3  3
 2  9  7 10
 2  2  9 19

Cmax: 20.0
Fbar:  9.0

Teste 3
maq #  p  C 
 0  1  6  6
 0  2  7 13
 0  0 12 25
 0  8 20 45
 1  4  2  2
 1 11 10 12
 1  9 13 25
 1 10 17 42
 2  5  3  3
 2  3  8 11
 2  6 15 26
 2  7 16 42

Cmax: 45.0
Fbar: 21.0

Teste 4
maq #  p  C 
 0  0  1  1
 0  5  6  7
 0  6  7 14
 0 11 12 26
 1  1  2  2
 1  4  5  7
 1  7  8 15
 1 10 11 26
 2  2  3  3
 2  3  4  7
 2  8  9 16
 2  9 10 26

Cmax: 26.0
Fbar: 12.5

Teste 5
maq #  p  C 
 0  0  1  1
 0  5  1  2
 0 10  1  3
 1  1  1  1
 1  6  1  2
 1 11  1  3
 2  2  1  1
 2  7  1  2
 2 12  1  3
 3  3  1  1
 3  8  1  2
 3 13  1  3
 4  4  1  1
 4  9  1  2
 4 14  1  3

Cmax:  3.0
Fbar:  2.0

Teste 6
maq #  p  C 
 0  0 13 13
 0 17 17 30
 0  8 48 78
 0  9 94 172
 1 13 14 14
 1  7 19 33
 1  1 48 81
 1  6 90 171
 2 15 15